In [44]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [45]:
### load the trained model ,scaler ,pickle,onehot
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [46]:
input_data  = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',	
    'Age':40,	
    'Tenure':3,	
    'Balance':60000,	
    'NumOfProducts':2,
    'HasCrCard':1,	
    'IsActiveMember':1,	
    'EstimatedSalary':50000	
}

In [47]:
print("model type:", type(model))
print("label_encoder_gender type:", type(label_encoder_gender))
print("onehot_encoder_geo type:", type(onehot_encoder_geo))
print("scaler type:", type(scaler))

model type: <class 'keras.src.models.sequential.Sequential'>
label_encoder_gender type: <class 'sklearn.preprocessing._label.LabelEncoder'>
onehot_encoder_geo type: <class 'sklearn.preprocessing._encoders.OneHotEncoder'>
scaler type: <class 'sklearn.preprocessing._data.StandardScaler'>


In [48]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\ml-pratice\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [49]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [50]:

input_data['Geography']

0    France
Name: Geography, dtype: object

In [51]:
input_data = pd.concat([input_data.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [52]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [53]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [54]:
## scaling the input data
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [56]:
## predicit churn
predition = model.predict(input_scaled)
predition

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


array([[0.03883593]], dtype=float32)

In [58]:
predition_proba = predition[0][0]
predition_proba

np.float32(0.038835928)

In [59]:
if predition_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
